<font color=teal>**SHORELINES WITH CRITERIA3**<font>

In [1]:
%load_ext autoreload
%autoreload

In [3]:
import os
import sys

from osgeo import osr, ogr, gdal
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import math
import time
import pandas as pd
from scipy import interpolate
import json
from matplotlib import colors as mcolors
import matplotlib.image as mpimg
import matplotlib
from shapely import geometry
import time 
import pyproj
from shapely.affinity import rotate
from shapely.geometry import Point, LineString, Polygon
plt.ion()
%matplotlib qt
matplotlib.use("Qt5Agg")

sys.path.append('/media/sf_VBox_Shared/PAOLA/PRISMA3/')
sys.path.append('/media/sf_VBox_Shared/PAOLA/PRISMA_FOAM/CODES/')

from PRISMA_SDS import PRIS_tools_v2, PRIS_img_v2, PRIS_profiles_v2, PRIS_Shoreline_v2
#import py3_0000_Functions as FC


In [4]:
def ProfileBounds(pr):
    
    """
    Returns the profile BB
    """
    
    xmin = pr.bounds['minx'].values[0]
    xmax = pr.bounds['maxx'].values[0]
    ymin = pr.bounds['miny'].values[0]
    ymax = pr.bounds['maxy'].values[0]
    
    return xmin, xmax, ymin, ymax

def ProfileBeginning(pr):
    
    """
    Gives the profile starting coordinate
    """
    
    pr.reset_index(inplace = True)
    coords = [(coords) for coords in list(pr.geometry[0].coords)]
    x1 = coords[0][0]
    y1 = coords[0][1]
    
    return x1, y1

def EuclideanDistance(x1, y1, x2, y2):
    
    """
    Calculates the euclidean distance between 2 points
    
    """
 
    dis_X = (x1 - x2)**2
    dis_Y = (y1 - y2)**2
    dist = np.sqrt(dis_X + dis_Y)

    return dist

def MidPointDistances(overlayed, x_prx, y_prx):
    
    """
    Obtain the midpoint of a segment
    
    """
    
    distances = []

    for point in overlayed['midpoint']:

        mid_point = list(point.coords)
        x2, y2 = mid_point[0][0], mid_point[0][1]

        distances.append(EuclideanDistance(x_prx, y_prx, x2 ,y2))
        
    overlayed['dist_midpoint'] = distances
    overlayed.sort_values(by = 'dist_midpoint', ascending = True, inplace = True)
    
    return overlayed

In [5]:
def read_list(s):
    
    return [float(x) for x in s[1:-1].replace(' ', '').split(',')]

def LoadMeanSpectra(path_csv):
    
    filename = 'ClassesMeanSS.csv'

    MeanSS = pd.read_csv(os.path.join(path_csv, filename), converters={'MeanSignature': read_list})
    
    return MeanSS

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array-value)).argmin()
    
    return array[idx], idx

def CalculateIndex(CW, arr): # rrow, ccol
    
    _, idx400 = find_nearest(CW, 400)
    _, idx700 = find_nearest(CW, 700)
    _, idx1000 = find_nearest(CW, 1000)
    _, idx1250 = find_nearest(CW, 1325)
    _, idx1500 = find_nearest(CW, 1500)

    img_index_up2 = np.nanmean(arr[:,:,int(idx400):int(idx700)], axis = 2)
    img_index_down2 = np.nanmean(arr[:,:,int(idx700):int(idx1250)], axis = 2)
    index2 = img_index_up2/img_index_down2
    
    return index2


In [6]:
def MeanReflectances(overlayed, arr):
    
    """
    Add to the overlated geodatabase the mean reflectance value of the particular pixel
    """
    
    mean_reflectance = []

    for r, c in zip(overlayed['row'], overlayed['col']):
        ss = arr[r,c,:]
        mean_reflectance.append(np.mean(ss))

    overlayed['mean_R'] = mean_reflectance
    
    return overlayed

def Interpolate_R_PR(overlayed, interval):
    
    x_old = np.array(overlayed['dist_midpoint'])
    y_old = np.array(overlayed['mean_R'])
    
    x_new = np.arange(x_old[0], x_old[-1], interval)
    f = interpolate.interp1d(x_old, y_old, kind = 'cubic')
    y_new =  f(x_new)
    
    deriv_new = y_new[1:] - y_new[:-1]
    idd_inter_new = np.argmin(deriv_new)
    inter = y_new[idd_inter_new]
    
    return idd_inter_new, inter, x_new, y_new, deriv_new

def SSInterfacePixe(GeoT, arr, lon, lat):

    # Pixel containing the transition coordinate
    
    col, row = PRIS_tools_v2.world_to_pixel(GeoT, lon, lat)
    
    # Get the spectral signature
    
    ss = arr[row, col, :]
    
    return col, row, ss
    
    

def PlotProfileValues(paths, date, idd_inter_new, inter, x_new, y_new, deriv_new, pr):

        
    pathS0 = os.path.join(paths['Interface_pix_plot'], 'MeanSS_pr')
    
    if not os.path.exists(pathS0):
        os.mkdir(pathS0)
    
    
    name = 'MeanSS_'+ pr + '_' + date + '.png'

    pathS = os.path.join(pathS0, date)

    if not os.path.exists(pathS):
        os.mkdir(pathS)


        
    plt.scatter(idd_inter_new, inter, s=150, c='firebrick', alpha = 0.5, label = 'interface')
    plt.plot(y_new, marker = 'o', color='royalblue', label = 'mean reflectance') # royalblue
    plt.plot(deriv_new, marker = 'o', color = 'slategray', label = 'derived') # #  skyblue

    ax.spines['top'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_color('grey')
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_color('grey')
    ax.grid(color='gainsboro', linestyle = '--', axis = 'y')

    plt.title('Profile '+ pr, fontsize = 15)
    plt.xlabel("d (m)", fontsize = 15)
    plt.ylabel("Mean Reflectance", fontsize = 15)
    plt.legend()
    plt.savefig(os.path.join(pathS, name))
    plt.show()
    plt.close()
    

In [7]:
def AzimuthAngle(coor_pr_i, coor_pr_f):
    angle = (180/np.pi) * math.atan2(coor_pr_f[0] - coor_pr_i[0], coor_pr_f[1] -coor_pr_i[1])
    
    if angle < 0:
        angle = (angle + 360)%360
    return(angle)

In [8]:
def dot(vA, vB):
    return vA[0]*vB[0] + vA[1] * vB[1]

def Cal_ang(lineA, lineB):
    vA = [(lineA[0][0] - lineA[1][0]), (lineA[0][1] - lineA[1][1])]
    vB = [(lineB[0][0] - lineB[1][0]), (lineB[0][1] - lineB[1][1])]

    # Get dot profuct
    dot_pro = dot(vA, vB)
    # Get magnitudes
    magA = dot(vA, vA)**0.5
    magB = dot(vB, vB)**0.5
    # Get cosine value
    cos_ = dot_pro/magA/magB
    
    # Get angle in radians and then convert to degrees
    if dot_pro == 0:

        ang_deg = np.degrees(np.arccos(0)) 
    else:
        angle = math.acos(dot_pro/magB/magA)
        # Doing angle <- angle mod 360
        ang_deg = math.degrees(angle)%360

    if ang_deg - 180 >= 0:
        return 360 - ang_deg
    else:
        return ang_deg

In [9]:
# This function is used only to check if a profiles is fully contained on the image BB

def ImageBB(paths, date):
    
    imgName = [f for f in os.listdir(paths['PanSharp_RGB']) if f.endswith('.tiff') and date in f][0]
    img = gdal.Open(os.path.join(paths['PanSharp_RGB'], imgName))

    nC = img.RasterXSize
    nR = img.RasterYSize

    geoT = img.GetGeoTransform()
    res = geoT[1]
    xmin = min(geoT[0], geoT[0] + nC * geoT[1])
    xmax = max(geoT[0], geoT[0] + nC * geoT[1])
    ymin = min(geoT[3], geoT[3] + nR * geoT[5])
    ymax = max(geoT[3], geoT[3] + nR * geoT[5])
    north = [(xmin, ymin), (xmin, ymax)]
    
    return xmin, xmax, ymin, ymax, north

<font color=red>**Here we define the functions needed to apply the first constrain to avoid the foam**<font>
    
    Mean of the medians of the curves obtained for the shorelines with and without foam in the training dataset

In [10]:
def Generate_Dict_Training_Dict(path_trainig):
    
    "Load the mean median vlues"
    
    info = pd.read_csv(os.path.join(path_trainig))
    groups = info.groupby('Proxy')
    dictio = {}
    for idd, group in groups:
        dictio[idd] = group['Medians'].quantile(q=0.9)
        
    return dictio


def turning_points(array):
    
    idx_max, idx_min = [], []
    if (len(array) < 3):
        return idx_min, idx_max
    NEUTRAL, RISING, FALLING = range(3)
    
    def get_state(a,b):
        if a < b: return RISING
        if a > b: return FALLING
        return NEUTRAL
    ps = get_state(array[0], array[1])
    begin = 1
    for i in range(2, len(array)):
        s = get_state(array[i-1], array[i])
        if s!= NEUTRAL:
            if ps != NEUTRAL and ps != s:
                if s == FALLING:
                    idx_max.append((begin + i - 1) // 2)
                else:
                    idx_min.append((begin + i - 1) // 2)
            begin = i
            ps = s
    return idx_min, idx_max

def MinimumPerSegment(indexes_turning, x_split, y_split):
    
    idds_inter = []
    inters_seg = []
    cols = []
    rows = []
    for a , b in zip(indexes_turning, indexes_turning[1:]):
        
        if b - a == 1:
            continue

        x_seg = x_split[a:b]
        y_seg = y_split[a:b]
        idd_inter_seg, inter_seg, x_new_seg, y_new_seg, deriv_new_seg = Interpolate_R_PR_Split(x_seg, y_seg)
        idds_inter.append(a + idd_inter_seg)
        inters_seg.append(inter_seg)
        
    return idds_inter, inters_seg

def Interpolate_R_PR_Split(x_split, y_split):
    
    x_new = x_split
    y_new = y_split
    
    #x_new = np.arange(x_old[0], x_old[-1])
    #f = interpolate.interp1d(x_old, y_old, kind = 'cubic')
    #y_new =  f(x_new)
    
    deriv_new = y_new[1:] - y_new[:-1]
    idd_inter_new = np.argmin(deriv_new)
    inter = y_new[idd_inter_new]
    
    return idd_inter_new, inter, x_new, y_new, deriv_new 


In [11]:
def Save_geojson(paths,shapefile, imagename, lon, lat):
    
    # Save into the geojson file
    
    geojson = PRIS_tools_v2.coor_to_geojson(lon, lat)
    
    filename = imagename.split('.')[0] + '_' + 'profilesFoam6_'+   '_'.join(shapefile.split('_')[2:4]) + '.geojon'
    
    dir_store = os.path.join(paths['SDS'], 'derivative')
    
    if not os.path.exists(dir_store):
        os.mkdir(dir_store)
    
    PRIS_tools_v2.save_geojson(dir_store, filename, geojson)

In [12]:
def Interpolate_Index_PR(overlayed, interval):
    
    x_old = np.array(overlayed['dist_midpoint'])
    y_old = np.array(overlayed['IndexFoam'])
    
    x_new = np.arange(x_old[0], x_old[-1], interval)
    f = interpolate.interp1d(x_old, y_old, kind = 'cubic')
    y_new =  f(x_new)
    
    overlayed = pd.DataFrame({'distances':x_new, 'IndexFoam':y_new})
    
    return overlayed

In [13]:
def SaveSubPixel_Txt(paths, shapefile ,imagename, lon, lat):
    
    dir_img =  os.path.join(paths['PanSharp_Square_Cropped'], imagename)
    raster = gdal.Open(dir_img)
    geoT = raster.GetGeoTransform()
    minx = geoT[0]
    maxy = geoT[3]
    
    upper_left = [minx, maxy]
    
    filename = imagename.split('.')[0] + '_' + 'profilesFoam6_'+   '_'.join(shapefile.split('_')[2:4]) + '.txt'
    
    dir_store = os.path.join(paths['SDS'], 'derivative')
    
    if not os.path.exists(dir_store):
        os.mkdir(dir_store)
        
    dir_fileout = os.path.join(paths['SDS'], 'derivative', filename)
    
    fileout = open(dir_fileout, "w")

    for lo, la in zip(lon, lat):
            
        sub_col = (lo - upper_left[0])/geoT[1]
        sub_row = (upper_left[1] - la)/geoT[1]
        fileout.write('{:9.3f} {:9.3f}\n'.format(sub_row, sub_col))
                         
    fileout.close()


In [11]:
def IndexValues(overlayed, index2):
    
    """
    Add to the overlated geodatabase the mean reflectance value of the particular pixel
    """
    
    index_values = []

    for r, c in zip(overlayed['row'], overlayed['col']):
        index_values.append(index2[r,c])

    overlayed['IndexFoam'] = index_values
    
    return overlayed

In [14]:
def AnaliziyngProfiles(profiles, date, grid, arr, img_BB, GeoT, plot):
    
    
    path00 = '/media/sf_VBox_Shared/PAOLA/PRISMA_FOAM/VALIDATING_SCENES'
    scene = 'FR'
    beach = 'FR01'
    img = 'PRS_L2D_STD_20211202103824_20211202103828_0001.he5'
    directories = PRIS_tools_v2.generate_structure(path00, scene, beach) 
    CW = PRIS_img_v2.ObtainCW(directories ,img)
    ratio = CalculateIndex(CW, arr)
    
    # Generate the DataFrame

    lon = []
    lat = []
    
    cols = []
    rows = []
    SS = []
    PRS = []
    
    #profiles = profiles.head(6)

    for pr in profiles['PR']:
        

        print("", end=f"\r Analyzing {pr}")
        
        # Search the profile

        prx = profiles[profiles['PR'] == pr]
        prx.reset_index(inplace = True, drop = True)
        
        if not img_BB.contains(prx['geometry'][0]):
            continue

        # Get pixel initial coordinate

        x_prx, y_prx = ProfileBeginning(prx)

        # Overlay the grid and the profiles geometries

        overlayed = gpd.overlay(prx, grid, how = 'intersection')
        overlayed.reset_index(inplace = True, drop = True)

        ## Check that the length of the obtained pixels is the same of the entire profile
        

        assert [np.abs(overlayed['geometry'].length.sum() - prx['geometry'].length) < 1.e-6][0][0]

        ## Check that all the distances are smaller than 5*sqrt(2)

        overlayed['length'] =  [line.length for line in overlayed['geometry']]
        overlayed['bool'] = overlayed['length'] < 5 * np.sqrt(2)
        

        assert overlayed['bool'].unique()[0]

        ## Obtain the mid point of the segments

        overlayed['midpoint'] = overlayed.apply(lambda row: row['geometry'].centroid, axis=1) #Find centroid

        ## Calculate the distances of each midpoint to the beginning profile point

        overlayed = MidPointDistances(overlayed, x_prx, y_prx)

        ## Add the mean reflectances
        
        overlayed = IndexValues(overlayed, ratio)
        overlayed = MeanReflectances(overlayed, arr)
        
        ## Add the distances
        
        overlayed['distances'] = overlayed['length'].cumsum()
        
        ## Interpolate distances an reflectances

        idd_inter_new, inter, x_new, y_new, deriv_new = Interpolate_R_PR(overlayed, 0.25)
        
        ## Interpolate also the Sand index
        
        index_frame = Interpolate_Index_PR(overlayed, 0.25)
        
        ## Extract the index foam
        
        SandIndex = index_frame['IndexFoam']
        
        # Check distance values
        
        assert list(x_new) == index_frame['distances'].to_list()
        
        ## Generate Dataframe
        
        b = deriv_new[-1]
        deriv_new2 = np.append(deriv_new,b)
        Interpolat = pd.DataFrame({'dis': x_new, 'mean_r':y_new, 'deriv':deriv_new2})
        
        ## Define which points are foam and which not
        
        thre = 0.12
        
        Interpolat['Bool_Foam'] = Interpolat['mean_R'] <= thre
        
        if len(Interpolat[Interpolat['Bool_Foam']]) == 0:
            continue
                
        # Cortar el Dataframe en el primer True
        
        Interpolat.sort_values(by = 'dis', ascending = True, inplace = True)
        
        # Get the position of the first True value
        
        cut_index = Interpolat[Interpolat['Bool_Foam']].index.min()
        
        if cut_index == 0:
            continue
        
        # Interpolated Sand
        
        Inter = Interpolat[0:cut_index]
        
        ## Get all the values before, which are supposed to be also sand
        
        assert np.unique(Inter['mean_R'] < thre) 
        
        ## Find the turning points of the derivative array
        
        deriv_new = Inter['deriv']
        idx_min, idx_max = turning_points(deriv_new)
        
        if len(idx_min)==0:
            continue
        
        ## Ahora tengo que ver cual es el minimo de los turning points minimos
        
        Turning_values = [Inter.iloc[int(f)]['deriv'] for f in idx_min]
        
        ff = pd.DataFrame({'values':Turning_values, 'idx':idx_min})
        
        
        ## Which is the inter value and where is located (needed to save the info)
        
        idd_inter_new = ff.sort_values(by=['values','idx'], ascending = True)['idx'].iloc[0]
        
        x_new = Inter['dis']
        
        new_length = x_new[idd_inter_new]

        # We have to know which is the profile angle to be able to 
        
        xmin_pr, xmax_pr, ymin_pr, ymax_pr = ProfileBounds(prx)
        
        
        north = [(xmin_pr, ymin_pr), (xmin_pr, ymax_pr)] # nortth direction
        
        coords = [(coords) for coords in list(prx.geometry[0].coords)]
        coor_pr_i, coor_pr_f = [coords[i] for i in (0,-1)]
        slp = (coor_pr_f[1] - coor_pr_i[1]) / (coor_pr_f[0] - coor_pr_i[0]) # slope profile
        pr_line = [(coor_pr_i[0], coor_pr_i[1]), (coor_pr_f[0], coor_pr_f[1])]
        
        angle = AzimuthAngle(coor_pr_i, coor_pr_f)
        Deg_rot = Cal_ang(north, pr_line)
        
        start = Point(coor_pr_i[0], coor_pr_i[1])
        end = Point(start.x  , start.y + new_length)
        Line = LineString([start, end])
        

        if (90 < angle <270) and (slp < 0):

            pr_new = rotate(Line , -Deg_rot , origin = [coor_pr_i[0], coor_pr_i[1]], use_radians = False)

        if (180 < angle <270) and (slp > 0):

            pr_new = rotate(Line , Deg_rot , origin = [coor_pr_i[0], coor_pr_i[1]], use_radians = False)

        if (0 < angle <90) and (slp > 0):

            pr_new = rotate(Line , -Deg_rot , origin = [coor_pr_i[0], coor_pr_i[1]], use_radians = False)
            
        if (270< angle < 360) and (slp < 0):
            
            pr_new = rotate(Line , Deg_rot , origin = [coor_pr_i[0], coor_pr_i[1]], use_radians = False)
        
        
        coords_n = [(coords) for coords in list(pr_new.coords)]
        coor_pr_iN, coor_pr_fN = [coords_n[i] for i in (0,-1)]
        
        col, row, ss = SSInterfacePixe(GeoT, arr, coor_pr_fN[0], coor_pr_fN[1])
        
        cols.append(col)
        rows.append(row)
        SS.append(list(ss))
        PRS.append(pr)

        lon.append(coor_pr_fN[0])
        lat.append(coor_pr_fN[1])
        
    interface_pix = pd.DataFrame({'pr':PRS, 'row':rows, 'col':cols, 'signature':SS})

    return lon, lat, interface_pix


In [15]:
def SubPixProfiles(paths,beach):
    
    # Load the images we have to analyze each one
    
    imgs = [f for f in os.listdir(paths['PanSharp_Square_Cropped']) if f.endswith('tiff')]
    
    
    for imagename in imgs:
        
        date = imagename.split('_')[1]
        
        if date == '20201015':
            
            continue
        
        print('Analyzing imgage: ', str(imagename), flush = True)
      
        
        dir_img =  os.path.join(paths['PanSharp_Square_Cropped'], imagename)
        print('leo imagen')
        arr, arr_rgb = PRIS_img_v2.numpy_from_tiff(dir_img, True)
        print('he leido imagen')
        img = gdal.Open(dir_img)
        GeoT = img.GetGeoTransform()
        proj = osr.SpatialReference(wkt=img.GetProjection())
        crs = pyproj.CRS("EPSG:" + proj.GetAttrValue('AUTHORITY',1))

        
        # CREATE THE RASTER GRID GEODATAFRAME
        
        #grid = GridRaster(dir_img, crs)

        dir_grids = os.path.join(paths['scene'] , beach, 'QGIS', 'Img_grid')
        gridName = [f for f in os.listdir(dir_grids) if date in f][0]
        grid = gpd.read_file(os.path.join(dir_grids, gridName))
        
        # Image Bounding Box
        
        xmin, xmax, ymin, ymax, north = ImageBB(paths, date)
        img_BB = Polygon([[xmin,ymin], [xmin, ymax], [xmax, ymax], [xmax, ymin]])
        
        ## LOAD THE PROFILES
        
        shapefiles = [f for f in os.listdir(paths['Profiles']) if f.endswith('.shp') and date in f]
        
        # if there is more than one profile per date (inclination study) we must iterate throught
        
        for shapefile in shapefiles:
            
            print('SDS for profiles: ', shapefile)
            
            filename = imagename.split('.')[0] + '_' + 'profiles_'+   '_'.join(shapefile.split('_')[2:4]) + '.txt'

            #if os.path.exists(os.path.join(paths['SDS'], 'derivative', filename)):
                #print('Already calculated')
                #continue
        
            profile_path = os.path.join(paths['Profiles'],shapefile) 

            if os.path.exists(profile_path):
                print('Load the profiles')
                profiles = gpd.read_file(profile_path, encoding="utf-8")
                crs_pr = profiles.crs
            else:
                print('No profiles')
                
            if '90_0' in shapefile:
                plot = False
            else:
                plot = False

            # ITERATE THROUGH THE PROFILES
            if '90_0' in shapefile:
                
                lon, lat, interface_pix = AnaliziyngProfiles(profiles,date, grid,arr, img_BB, GeoT, plot)
            
            # SAVE THE INTERFACE_PIX DATAFRAME
            
            # This information is saved only for the k-means shoreline (perp pr)
            
            if '90_0' in shapefile:
                
                name_csv = 'InterfacePix_' + date +'_'+('_').join(shapefile.split('.')[0].split('_')[-3:-1]) + '.csv'
        
                interface_pix.to_csv(os.path.join(paths['Interface_pix_csv'], name_csv), index = False)

            # SAVE THE SHORELINE

            Save_geojson(paths,shapefile, imagename, lon, lat)
            SaveSubPixel_Txt(paths,shapefile, imagename, lon, lat)


In [17]:
path0 = '/media/sf_VBox_Shared/PAOLA/PRISMA_FOAM/VALIDATING_SCENES/'
scene = 'IT'

beaches = [f for f in os.listdir(os.path.join(path0, 'SCENES', scene)) if 'IT05' in f] #if '01' in f

#path_csv = '/media/sf_VBox_Shared/PAOLA/PRISMA_FOAM/RESULTS/csv/'
#MeanSS = LoadMeanSpectra(path_csv)

for beach in beaches:
    
    print(beach)
    
    paths = PRIS_tools_v2.generate_structure(path0, scene, beach)
    
    dir_inter_Rat_txtPlot = os.path.join(paths['scene'], beach, 'Results', 'Interface_pix', 'txtPlot')
    
    if not os.path.exists(dir_inter_Rat_txtPlot):
        os.makedirs(dir_inter_Rat_txtPlot)
        
    
    paths['Interface_pix_txtPlot'] = dir_inter_Rat_txtPlot
    
    SubPixProfiles(paths, beach)

IT05
Analyzing imgage:  IT_20220201_PRISMA_PANS_0000_0000_00000_00000.tiff
leo imagen
he leido imagen
SDS for profiles:  IT_20220201_-90_0_pr.shp
Load the profiles
 Analyzing PR118

array([ True])

In [180]:

imgs = [f for f in os.listdir(paths['PanSharp_Square_Cropped']) if f.endswith('tiff')]
    
    
for imagename in imgs:

    date = imagename.split('_')[1]

    if date == '20201015':

        continue

    print('Analyzing imgage: ', str(imagename), flush = True)


    dir_img =  os.path.join(paths['PanSharp_Square_Cropped'], imagename)
    print('leo imagen')
    arr, arr_rgb = PRIS_img_v2.numpy_from_tiff(dir_img, True)
    print('he leido imagen')
    img = gdal.Open(dir_img)
    GeoT = img.GetGeoTransform()
    proj = osr.SpatialReference(wkt=img.GetProjection())
    crs = pyproj.CRS("EPSG:" + proj.GetAttrValue('AUTHORITY',1))


    # CREATE THE RASTER GRID GEODATAFRAME

    #grid = GridRaster(dir_img, crs)

    dir_grids = os.path.join(paths['scene'] , beach, 'QGIS', 'Img_grid')
    gridName = [f for f in os.listdir(dir_grids) if date in f][0]
    grid = gpd.read_file(os.path.join(dir_grids, gridName))

    # Image Bounding Box

    xmin, xmax, ymin, ymax, north = ImageBB(paths, date)
    img_BB = Polygon([[xmin,ymin], [xmin, ymax], [xmax, ymax], [xmax, ymin]])

    ## LOAD THE PROFILES

    shapefiles = [f for f in os.listdir(paths['Profiles']) if f.endswith('.shp') and date in f]

    # if there is more than one profile per date (inclination study) we must iterate throught

    for shapefile in shapefiles:

        print('SDS for profiles: ', shapefile)

        filename = imagename.split('.')[0] + '_' + 'profiles_'+   '_'.join(shapefile.split('_')[2:4]) + '.txt'

        #if os.path.exists(os.path.join(paths['SDS'], 'derivative', filename)):
            #print('Already calculated')
            #continue

        profile_path = os.path.join(paths['Profiles'],shapefile) 

        if os.path.exists(profile_path):
            print('Load the profiles')
            profiles = gpd.read_file(profile_path, encoding="utf-8")
            crs_pr = profiles.crs
        else:
            print('No profiles')



Analyzing imgage:  SPI_20201115_PRISMA_PANS_0050_0918_00084_00078.tiff
leo imagen
he leido imagen
SDS for profiles:  SPI_20201115_-90_0_pr.shp
Load the profiles


In [240]:
o = 0
for pr in profiles['PR']:
        

        print("", end=f"\r Analyzing {pr}")
        
        # Search the profile

        prx = profiles[profiles['PR'] == pr]
        prx.reset_index(inplace = True, drop = True)
        
        if not img_BB.contains(prx['geometry'][0]):
            continue

        # Get pixel initial coordinate

        x_prx, y_prx = ProfileBeginning(prx)

        # Overlay the grid and the profiles geometries

        overlayed = gpd.overlay(prx, grid, how = 'intersection')
        
        o += 1
        
        if o == 1:
        
            break

 Analyzing PR1

In [241]:
overlayed['length'] =  [line.length for line in overlayed['geometry']]
overlayed['bool'] = overlayed['length'] < 5 * np.sqrt(2)

In [242]:
        ## Obtain the mid point of the segments

overlayed['midpoint'] = overlayed.apply(lambda row: row['geometry'].centroid, axis=1) #Find centroid

        ## Calculate the distances of each midpoint to the beginning profile point

overlayed = MidPointDistances(overlayed, x_prx, y_prx)

        ## Add the mean reflectances
        

overlayed = MeanReflectances(overlayed, arr)

In [243]:
overlayed['distances'] = overlayed['length'].cumsum()

In [244]:
path00 = '/media/sf_VBox_Shared/PAOLA/PRISMA_FOAM/VALIDATING_SCENES'
scene = 'FR'
beach = 'FR01'
img = 'PRS_L2D_STD_20211202103824_20211202103828_0001.he5'
directories = PRIS_tools_v2.generate_structure(path00, scene, beach) 
CW = PRIS_img_v2.ObtainCW(directories ,img)
ratio = CalculateIndex(CW, arr)

In [247]:
overlayed = MidPointDistances(overlayed, x_prx, y_prx)

## Add the mean reflectances

overlayed = IndexValues(overlayed, ratio)
overlayed = MeanReflectances(overlayed, arr)

## Add the distances

overlayed['distances'] = overlayed['length'].cumsum()

## Interpolate distances an reflectances

idd_inter_new, inter, x_new, y_new, deriv_new = Interpolate_R_PR(overlayed, 0.25)

## Interpolate also the Sand index

index_frame = Interpolate_Index_PR(overlayed, 0.25)

## Extract the index foam

SandIndex = index_frame['IndexFoam']

# Check distance values

assert list(x_new) == index_frame['distances'].to_list()

## Generate Dataframe

b = deriv_new[-1]
deriv_new2 = np.append(deriv_new,b)
Interpolat = pd.DataFrame({'dis': x_new, 'mean_r':y_new, 'deriv':deriv_new2, 'IndexFoam': SandIndex})

## Define which points are foam and which not

thre = 0.75

Interpolat['Bool_Foam'] = Interpolat['IndexFoam'] >= 0.74

if len(Interpolat[Interpolat['Bool_Foam']]) == 0:
    print('GGG')

# Cortar el Dataframe en el primer True

Interpolat.sort_values(by = 'dis', ascending = True, inplace = True)

# Get the position of the first True value

cut_index = Interpolat[Interpolat['Bool_Foam']].index.min()

# Interpolated Sand

Inter = Interpolat[0:cut_index]

## Get all the values before, which are supposed to be also sand

#assert np.unique(Inter['IndexFoam'] < 0.74) 

## Find the turning points of the derivative array

deriv_new = Inter['deriv']
idx_min, idx_max = turning_points(deriv_new)


## Ahora tengo que ver cual es el minimo de los turning points minimos

Turning_values = [Inter.iloc[int(f)]['deriv'] for f in idx_min]

#ff = pd.DataFrame({'values':Turning_values, 'idx':idx_min})


## Which is the inter value and where is located (needed to save the info)

#idd_inter_new = ff.sort_values(by=['values','idx'], ascending = True)['idx'].iloc[0]


In [251]:
idx_max

[]

In [233]:
Interpolat[Interpolat['Bool_Foam']].index.min()

0

In [203]:
Interpolat[Interpolat['Bool_Foam'] == False]

,dis,mean_r,deriv,IndexFoam,Bool_Foam
94,25.182894,0.143007,0.002403,0.714659,False
95,25.432894,0.145410,0.002003,0.622150,False
96,25.682894,0.147413,0.001581,0.521967,False
97,25.932894,0.148994,0.001138,0.423896,False
98,26.182894,0.150132,0.000674,0.337724,False
...,...,...,...,...,...
246,63.182894,0.018469,0.000140,-35.985544,False
247,63.432894,0.018608,0.000112,-27.836845,False
248,63.682894,0.018721,0.000084,-19.819446,False
249,63.932894,0.018805,0.000057,-12.169750,False


In [157]:
Interpolat.sort_values(by = 'dis', ascending = True)

,dis,mean_r,deriv,IndexFoam,Bool_Foam
0,0.113637,0.148846,-0.000195,0.712426,False
1,0.363637,0.148651,-0.000184,0.713738,False
2,0.613637,0.148467,-0.000175,0.714972,False
3,0.863637,0.148292,-0.000166,0.716132,False
4,1.113637,0.148127,-0.000157,0.717222,False
...,...,...,...,...,...
466,116.613637,0.032265,0.000241,2.391407,True
467,116.863637,0.032506,0.000254,2.270090,True
468,117.113637,0.032761,0.000268,2.121036,True
469,117.363637,0.033029,0.000284,1.942544,True


In [204]:
overlayed = IndexValues(overlayed, index2)
overlayed = MeanReflectances(overlayed, arr)

In [205]:
## Which points are foam and which are not

overlayed['Foam_bool'] = overlayed['IndexFoam'] >= 0.74

In [252]:
foam = Interpolat[Interpolat['Bool_Foam']]
no_foam = Interpolat[~Interpolat['Bool_Foam']]

In [253]:
idd_inter_new, inter, x_new, y_new, deriv_new = Interpolate_R_PR(overlayed, 0.25)

In [216]:
#overlayed['distances'] = overlayed['length'].cumsum()

In [239]:
no_foam

,dis,mean_r,deriv,IndexFoam,Bool_Foam
94,25.182894,0.143007,0.002403,0.714659,False
95,25.432894,0.145410,0.002003,0.622150,False
96,25.682894,0.147413,0.001581,0.521967,False
97,25.932894,0.148994,0.001138,0.423896,False
98,26.182894,0.150132,0.000674,0.337724,False
...,...,...,...,...,...
246,63.182894,0.018469,0.000140,-35.985544,False
247,63.432894,0.018608,0.000112,-27.836845,False
248,63.682894,0.018721,0.000084,-19.819446,False
249,63.932894,0.018805,0.000057,-12.169750,False


In [254]:
with plt.rc_context({'axes.edgecolor':'grey', 'xtick.color':'grey', 'ytick.color':'grey'}):


    fig, axs = plt.subplots(3,1 ,figsize=(9,12), sharex = True) #figsize=(6*1.618,6)
    
    l0 = axs[0].hlines(1, 0, x_new[-1], linestyles = '--', label = 'Foam threshold')
    axs[0].plot(overlayed['distances'], overlayed['IndexFoam'], linestyle = '-', linewidth = 2, c = 'skyblue')
    l1, = axs[1].plot(x_new, y_new, linestyle = '-', linewidth = 2, color='royalblue', label = 'mean reflectance') # royalblue
    l2, = axs[2].plot(x_new[:-1], deriv_new, linestyle = '-', linewidth = 2, color = 'slategray',
                      label = 'derivative') # #  skyblue
    
    l3 = axs[1].scatter(foam['dis'], foam['mean_r'], s = 60, alpha = 0.4, c = 'darkred', label = 'Foam ', zorder = 10)
    l4 = axs[1].scatter(no_foam['dis'], no_foam['mean_r'], s = 60, alpha = 0.4, label = 'Not foam ', c = 'darkseagreen', zorder = 10)
    #l3 = axs[1].scatter(x_new[idd_inter_new], inter, s=120, alpha = 0.6, c ='firebrick', label = 'Interface', zorder = 10)
    axs[2].scatter(x_new[idd_inter_new], deriv_new[idd_inter_new], s=120, alpha = 0.6, c='firebrick', zorder = 10)
    
    o = 4
    
    #for diss in distances[0]:
    axs[0].axvline(distance, color= 'gainsboro', ls = 'dashed', linewidth = 3, zorder = 0)
    axs[1].axvline(distance, color= 'gainsboro', ls = 'dashed', linewidth = 3, zorder = 0)
    axs[2].axvline(distance, color = 'gainsboro', ls = 'dashed', linewidth = 3, zorder = 0)
        #o = o+1
    
    
    axs[0].spines['top'].set_visible(False)
    axs[0].spines['top'].set_visible(False)
    axs[0].spines['bottom'].set_color('grey')
    axs[0].spines['right'].set_visible(False)
    axs[0].spines['left'].set_color('grey')
    
    axs[1].spines['top'].set_visible(False)
    axs[1].spines['top'].set_visible(False)
    axs[1].spines['bottom'].set_color('grey')
    axs[1].spines['right'].set_visible(False)
    axs[1].spines['left'].set_color('grey')
    
    axs[2].spines['top'].set_visible(False)
    axs[2].spines['top'].set_visible(False)
    axs[2].spines['bottom'].set_color('grey')
    axs[2].spines['right'].set_visible(False)
    axs[2].spines['left'].set_color('grey')
    
        
    
    #axs[0].set_xticks([])
    axs[1].axes.get_xaxis().set_visible(False)
    axs[0].set_ylabel(r'$I_{Foam}$', fontsize = 20)
    axs[2].set_ylabel(r"$\Delta \overline{R}/\Delta d$", fontsize = 20)
    axs[1].set_ylabel(r"$\overline{R}$", fontsize = 20)

    # axs[0].grid(color='gainsboro', linestyle = '--', axis = 'y')
    plt.xlabel("d (m)", fontsize = 20)
    
    plt.subplots_adjust(hspace = 0.01)
    axs[1].legend(handles = [ l3, l4], loc = 'upper left', fontsize = 18,
                 borderpad = 0.05,
                 labelspacing = 0.1, handletextpad = 0.1)
    
    axs[0].legend(handles = [ l0], loc = 'upper right', fontsize = 18,
             borderpad = 0.05,
             labelspacing = 0.1, handletextpad = 0.1)
    
    

In [129]:
# Trova quall'è il primo punto di intersection between the horizontal threshold and the curve

pos = []
pos.append([np.where(overlayed['IndexFoam']==f) for f in overlayed['IndexFoam'] if f>=0.74])
pos = pos[0]
distance = overlayed.iloc[pos[0]]['distances'].values

In [137]:
overlayed[overlayed['Foam_bool']]

,index,PR,row,col,geometry,length,bool,midpoint,dist_midpoint,mean_R,IndexFoam,Foam_bool,distances
24,0,PR1,3,40,"LINESTRING (295335.227 3880842.856, 295335.000...",0.227275,True,POINT (295335.1136363638 3880842.855580793),0.113637,0.148846,30.948641,True,0.227275


In [132]:
plt.imshow(ratio, vmin = -1, vmax = 3)
plt.colorbar()

In [165]:
ff

,values,idx
